# LCEL

## Basic example: prompt + model + output parser

https://python.langchain.com/docs/expression_language/get_started

In [1]:
from load_model import *

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [5]:
model = llm

In [9]:
# from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

prompt = ChatPromptTemplate.from_template("tell me a possible cause of the diagnostic trouble {trouble} in the process of car-making")
# model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

print(chain.invoke({"trouble": "battery"}))

There are several possible causes of diagnostic trouble in the process of car-making, including:

1. Worn or damaged battery: Over time, the battery in your car can become worn out or damaged, causing it to not hold a charge properly.
2. Dirty or corroded battery connector: A dirty or corroded battery connector can prevent the battery from receiving the proper charge.
3. Leakage in the battery harness: A leakage in the battery harness can cause the battery to become damaged or cause it to not hold a charge properly.
4. Worn or damaged starter motor: A worn or damaged starter motor can cause it to not start the car properly, causing a diagnostic trouble.
5. Worn or damaged alternator or starter motor wiring: Worn or damaged wiring in the alternator or starter motor can cause it to not work properly, causing a diagnostic trouble.
6. Worn or damaged A/C compressor: A worn or damaged A/C compressor can cause it to not work properly, causing a diagnostic trouble.
7. Worn or damaged brakes: 

### 1. Prompt

In [10]:
prompt_value = prompt.invoke({"trouble": "battery"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a possible cause of the diagnostic trouble battery in the process of car-making')])

In [11]:
prompt_value.to_messages()

[HumanMessage(content='tell me a possible cause of the diagnostic trouble battery in the process of car-making')]

In [12]:
prompt_value.to_string()

'Human: tell me a possible cause of the diagnostic trouble battery in the process of car-making'

### 2. Model

In [13]:
message = model.invoke(prompt_value)
message

"There are several possible causes of diagnostic trouble in the process of car-making, including:\n\n1. Worn or damaged battery: The battery in a car is a critical component that powers the starter and lights. If the battery is worn, damaged, or not functioning properly, it can cause the car to start trouble.\n2. Leakage of fluid: The fluid in the car's battery is used to lubricate the moving parts and keep the battery in good working condition. If the fluid leaks, it can cause problems with the battery's performance.\n3. Overheating: When the battery in a car heats up too much, it can cause problems with the electrical system in the car.\n4. short circuits: Short circuits occur when electrical wires are connected directly to the battery, rather than the battery's positive and negative terminals. This can cause the battery to overheat and can also cause the electrical system to malfunction.\n5. Connector problems: The connectors in the battery can become loose or damaged, causing an in

### 3. Output Parser

In [14]:
m = output_parser.invoke(message)
print(m)

There are several possible causes of diagnostic trouble in the process of car-making, including:

1. Worn or damaged battery: The battery in a car is a critical component that powers the starter and lights. If the battery is worn, damaged, or not functioning properly, it can cause the car to start trouble.
2. Leakage of fluid: The fluid in the car's battery is used to lubricate the moving parts and keep the battery in good working condition. If the fluid leaks, it can cause problems with the battery's performance.
3. Overheating: When the battery in a car heats up too much, it can cause problems with the electrical system in the car.
4. short circuits: Short circuits occur when electrical wires are connected directly to the battery, rather than the battery's positive and negative terminals. This can cause the battery to overheat and can also cause the electrical system to malfunction.
5. Connector problems: The connectors in the battery can become loose or damaged, causing an internal 

### 4. Entire Pipeline

In [15]:
input = {"trouble": "battery"}

prompt.invoke(input)

ChatPromptValue(messages=[HumanMessage(content='tell me a possible cause of the diagnostic trouble battery in the process of car-making')])

In [16]:
(prompt | model).invoke(input)

"There are several possible causes of diagnostic trouble in the process of car-making, including:\n\n1. Worn or damaged battery: Over time, the battery in your car's diagnostic trouble system may wear out or become damaged, causing it to malfunction.\n2. Wired incorrectly: If the wires in your car's diagnostic trouble system are not properly connected to the battery or the rest of the system, it can cause problems with the system functioning.\n3. Leakage: If there is a small leakage of battery juice into the diagnostic trouble system, it can cause the system to malfunction.\n4. Overheating: If the battery in your car's diagnostic trouble system overheats, it can cause problems with the system functioning.\n5. Worming: If a foreign object, such as a worm, comes into contact with the battery or the diagnostic trouble system, it can cause problems with the system functioning.\n\nIt's important to note that these are just a few possible causes of diagnostic trouble in the process of car-ma

## RAG Search Example

https://python.langchain.com/docs/integrations/text_embedding/sagemaker-endpoint

In [19]:
import json
from typing import Dict, List

from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.embeddings.sagemaker_endpoint import EmbeddingsContentHandler

In [23]:
class ContentHandler(EmbeddingsContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, inputs: list[str], model_kwargs: Dict) -> bytes:
        """
        Transforms the input into bytes that can be consumed by SageMaker endpoint.
        Args:
            inputs: List of input strings.
            model_kwargs: Additional keyword arguments to be passed to the endpoint.
        Returns:
            The transformed bytes input.
        """
        # Example: inference.py expects a JSON string with a "inputs" key:
        input_str = json.dumps({"inputs": inputs, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> List[List[float]]:
        """
        Transforms the bytes output from the endpoint into a list of embeddings.
        Args:
            output: The bytes output from SageMaker endpoint.
        Returns:
            The transformed output - list of embeddings
        Note:
            The length of the outer list is the number of input strings.
            The length of the inner lists is the embedding dimension.
        """
        # Example: inference.py returns a JSON string with the list of
        # embeddings in a "vectors" key:
        response_json = json.loads(output.read().decode("utf-8"))
        # return response_json["vectors"]
        return response_json

content_handler = ContentHandler()

In [24]:
embeddings = SagemakerEndpointEmbeddings(
    # credentials_profile_name="credentials-profile-name",
    endpoint_name=ENDPOINT_NAME,
    region_name=REGION,
    content_handler=content_handler,
)

# client = boto3.client(
#     "sagemaker-runtime",
#     region_name="us-west-2"
# )
# embeddings = SagemakerEndpointEmbeddings(
#     endpoint_name="huggingface-pytorch-inference-2023-03-21-16-14-03-834",
#     client=client
#     content_handler=content_handler,
# )

In [26]:
# # from langchain.chat_models import ChatOpenAI
# # from langchain.embeddings import OpenAIEmbeddings
# from langchain.prompts import ChatPromptTemplate
# from langchain.schema.output_parser import StrOutputParser
# from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
# from langchain.vectorstores import DocArrayInMemorySearch

# vectorstore = DocArrayInMemorySearch.from_texts(
#     ["harrison worked at kensho", "bears like to eat honey"],
#     # embedding=OpenAIEmbeddings(),
#     embedding=embeddings,
# )

# retriever = vectorstore.as_retriever()

In [27]:
# template = """Answer the question based only on the following context:
# {context}

# Question: {question}
# """
# prompt = ChatPromptTemplate.from_template(template)
# model = ChatOpenAI()
# output_parser = StrOutputParser()

# setup_and_retrieval = RunnableParallel(
#     {"context": retriever, "question": RunnablePassthrough()}
# )
# chain = setup_and_retrieval | prompt | model | output_parser

# chain.invoke("where did harrison work?")